# Extraindo informações sobre as turmas
Esse notebook tem como objetivo utilizar os datasets de matriculas e turmas para pegar gerar informações a respeito das turmas.

In [3]:
import pandas as pd
import numpy as np

In [4]:
matriculas = pd.read_csv("data/matriculas-de-2016.2.csv", sep=";")

In [5]:
# Pegando apenas as turmas de CeT
matriculas_cet = matriculas[matriculas.id_curso == 10320810]

In [6]:
matriculas_cet.descricao.unique()

array(['CANCELADO', 'APROVADO', 'TRANCADO',
       'REPROVADO POR MÉDIA E POR FALTAS', 'APROVADO POR NOTA',
       'INDEFERIDO', 'DESISTENCIA', 'REPROVADO', 'REPROVADO POR NOTA',
       'EXCLUIDA', 'REPROVADO POR NOTA E FALTA', 'REPROVADO POR FALTAS'],
      dtype=object)

In [7]:
# Pegar apenas quem começou a pagar a materia

indesejada = ['DESISTENCIA', 'INDEFERIDO']

matriculas_cet = matriculas_cet[~matriculas_cet.descricao.isin(indesejada)]

In [8]:
# matriculas_cet[matriculas_cet.descricao == 'CANCELADO']

In [9]:
matriculas_cet.shape

(64153, 11)

In [10]:
matriculas_cet.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

In [11]:
novo = matriculas_cet.drop_duplicates(subset=['id_turma', 'discente'])
novo = novo.dropna(subset=['media_final', 'numero_total_faltas'])

In [12]:
novo.shape
novo.head()

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,Unnamed: 10
380,57586700,1470770112749dc578c39fb86d139215,10320810.0,1.0,8.7,f,0.0,7.9,4.0,APROVADO,NaN
386,57586700,6b8e7705214ba419989eb54ca71ad283,10320810.0,1.0,0.0,f,0.0,0.0,54.0,CANCELADO,NaN
389,57586700,466406941f0280b618af91a513365eeb,10320810.0,1.0,4.7,f,0.0,3.9,12.0,CANCELADO,NaN
392,57586700,fa33216eec66570333dcf92214b5cebc,10320810.0,1.0,3.6,f,0.0,2.3,36.0,REPROVADO POR MÉDIA E POR FALTAS,NaN
395,57586700,7f3f3f94469b917f27ca1797af4bf542,10320810.0,1.0,9.9,f,0.0,8.0,6.0,APROVADO,NaN


In [13]:
novo.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

## Extraindo as informações estatísticas

In [14]:
# Calculando qnt_disc
qnt_disc = novo.groupby('id_turma', as_index=False)['media_final'].count() \
                    .rename(columns={'media_final': 'qnt_disc'})

# Calculando media_turma
media_turma = novo.groupby('id_turma', as_index=False)['media_final'].mean() \
                    .rename(columns={'media_final': 'media_turma'})

# Calculando media_faltas
media_faltas = novo.groupby('id_turma', as_index=False)['numero_total_faltas'].mean() \
                    .rename(columns={'numero_total_faltas': 'media_faltas'})

# Calculando qnt_reposicao
qnt_reposicao = novo[novo.reposicao == 't'].groupby('id_turma', as_index=False)['reposicao'].count() \
                    .rename(columns={'reposicao': 'qnt_reposicao'})

# Calculando qnt_aprovados
aprovados = ['APROVADO', 'APROVADO POR NOTA']
qnt_aprovados = novo[novo.descricao.isin(aprovados)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_aprovados'})

# Calculando qnt_reprovados
reprovado = ['REPROVADO', 'REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 
             'REPROVADO POR NOTA E FALTA', 'REPROVADO POR MÉDIA E POR FALTAS']
qnt_reprovados = novo[novo.descricao.isin(reprovado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_reprovados'})

# Calculando qnt_trancamento
trancado = ['CANCELADO', 'EXCLUIDA', 'TRANCADO']
qnt_trancamento = novo[novo.descricao.isin(trancado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_trancamento'})

In [15]:
# mesclando os datasets
turmas = pd.merge(left=qnt_disc, right=qnt_aprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_reprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_trancamento, how='outer')

In [16]:
turmas[turmas.qnt_disc > 100].head()

,id_turma,qnt_disc,qnt_aprovados,qnt_reprovados,qnt_trancamento
101,57585662,125,54.0,61.0,10.0
102,57585665,113,45.0,65.0,3.0
104,57585668,102,69.0,31.0,2.0
111,57585689,126,113.0,13.0,NaN
115,57585695,132,56.0,68.0,8.0


In [17]:
# Realizando o merge com as demais informações
turma_geral = pd.merge(left=turmas, right=media_turma, how='outer')
turma_geral = pd.merge(left=turma_geral, right=media_faltas, how='outer')
turma_geral = pd.merge(left=turma_geral, right=qnt_reposicao, how='outer')

# Preenchendo os valores vazios com 0
turma_geral = turma_geral.fillna(value=0)

In [18]:
turma_geral.to_csv('outputs/turmas_dados_2016_2.csv', sep=';', index=False)